# Графики

In [ ]:
from tqdm import tqdm
quant_hist = 2

data = dataset.interactions.df

# создадим 2 сплита 1 день - 60 день и 1 день - 120 день
splitter = TimeRangeSplitter(
    test_size="60D",
    n_splits=quant_hist,
    filter_already_seen=True,
)

splits = splitter.get_test_fold_borders(dataset.interactions)

first_date = splits[0][0]

sns.set_style("whitegrid")
sns.set_palette("bright")

fig, ax = plt.subplots(figsize=(10, 6))

als_ = ImplicitALSWrapperModel(AlternatingLeastSquares(factors=32, num_threads=7))

recoms_list = []
for i in tqdm(range(quant_hist)):
    second_date = splits[i][1]
    # Фильтруем данные по splitter dates
    als_df = data[(pd.to_datetime(data['datetime']) >= pd.to_datetime(first_date)) & (pd.to_datetime(data['datetime']) <= pd.to_datetime(second_date))]
    als_dataset =  Dataset.construct(
    interactions_df=als_df,
    )
    als_model = als_.fit(als_dataset)
    recoms = als_model.recommend(users=als_dataset.user_id_map.external_ids, dataset=als_dataset, filter_viewed=True, k=100)
    print(recoms["score"].describe())
    recoms_list.append(recoms["score"])

    sns.histplot(recoms["score"], ax=ax, label=f'Распределение скоров {i}',  kde=True, bins=50) # Добавлен kde=True для сглаживания



ax.set_title('Несколько гистограмм на одном графике')
ax.set_xlabel('Значения')
ax.set_ylabel('Частота')
ax.legend()

# Отображаем график
plt.show()


# Нормировка

In [ ]:
# Добавим нормировку
def normalize(data):
    return (data - data.mean()) / data.std()

recom_sasrec["sasrec_score"] = normalize(recom_sasrec["sasrec_score"])
recom_ease["ease_score"] = normalize(recom_ease["ease_score"])
recom_als["als_score"] = normalize(recom_als["als_score"])

# SHAP

In [ ]:
import shap

# Создайте экземпляр SHAP
explainer = shap.TreeExplainer(model)

# Получите SHAP-значения для данных
shap_values = explainer(X_val.drop(columns=["user_id", "item_id"]))

# Постройте SHAP-график
shap.plots.beeswarm(shap_values)


# Calc NDCG

In [ ]:
from sklearn.metrics import ndcg_score
import numpy as np
from tqdm import tqdm

true_relevance = pd.read_csv("data/true_relevance.csv")

val_scores = X_val[["user_id", "item_id"]]
val_scores['prediction'] = model.predict(val_pool)

all_scores = val_scores.merge(true_relevance, on=["user_id", "item_id"])

all_ndcg = []

for user in tqdm(all_scores["user_id"].unique()):
    if all_scores[all_scores["user_id"] == user].shape[0] > 1:
        sc = ndcg_score(all_scores.loc[all_scores["user_id"] == user, "true_relevance"].to_numpy().reshape(1, -1), all_scores.loc[all_scores["user_id"] == user, "prediction"].to_numpy().reshape(1, -1), k=6)
        all_ndcg.append(sc)

print(np.mean(all_ndcg))

# Правильный датасет

In [ ]:
# Обучаемся - на логах
target_df = interactions_second_period[["user_id", "item_id"]].drop_duplicates()
target_df["target"] = interactions_second_period["action"].map({"click": 1, "show": 0})


all_features = pd.merge(users["user_id"], items["item_id"], how="cross")
all_features = all_features.merge(target_df, on=["user_id", "item_id"], how="left")
all_features = all_features.fillna(0)
